IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M6\NO_FAULT6


In [3]:
guasto=0

VOLO m6 FAULT 10%

In [4]:
rcou_m6_nofault = pd.read_csv("RCOU.csv")
rcou_m6_nofault = rcou_m6_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m6_nofault = rcou_m6_nofault[((rcou_m6_nofault['C9'] >= 1350) & (rcou_m6_nofault['C10'] >= 1350) & (rcou_m6_nofault['C11'] >= 1350) & (rcou_m6_nofault['C12'] >= 1350) & (rcou_m6_nofault['C13'] >= 1350) & (rcou_m6_nofault['C14']>= 1350))]
rcou_m6_nofault=rcou_m6_nofault.reset_index(drop=True)
display(rcou_m6_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,73585436,1377,1436,1362,1450,1416,1400
1,73609813,1415,1428,1402,1439,1470,1366
2,73620421,1415,1438,1423,1431,1483,1364
3,73663445,1448,1452,1468,1431,1530,1356
4,73673952,1451,1461,1462,1449,1523,1379
...,...,...,...,...,...,...,...
5481,141114087,1562,1440,1526,1481,1478,1529
5482,141125072,1574,1422,1525,1479,1490,1516
5483,141136462,1603,1378,1547,1451,1544,1454
5484,141345328,1405,1392,1399,1398,1360,1435


In [5]:
min=rcou_m6_nofault['TimeUS'][0]
max=rcou_m6_nofault['TimeUS'][len(rcou_m6_nofault)-1]
print(max)
print(min)

141376898
73585436


In [6]:
xkf1_m6_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m6_nofault = xkf1_m6_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m6_nofault = xkf1_m6_nofault[((xkf1_m6_nofault['TimeUS'] >= min) & (xkf1_m6_nofault['TimeUS'] <= max	))]
xkf1_m6_nofault = xkf1_m6_nofault.reset_index(drop=True)
print(xkf1_m6_nofault)

         TimeUS  Roll  Pitch    Yaw
0      73597012  1.77  -1.55  15.93
1      73607972  1.70  -1.53  15.96
2      73617984  1.62  -1.51  15.99
3      73628863  1.62  -1.52  16.00
4      73638809  1.63  -1.51  16.01
...         ...   ...    ...    ...
6139  141325367 -0.09  -2.57   4.16
6140  141343293 -0.16  -2.48   3.98
6141  141353420 -0.13  -2.25   3.91
6142  141364373 -0.15  -2.06   3.89
6143  141374850 -0.24  -1.93   3.86

[6144 rows x 4 columns]


In [7]:
att_m6_nofault = pd.read_csv("ATT.csv")
att_m6_nofault = att_m6_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m6_nofault = att_m6_nofault[((att_m6_nofault['TimeUS'] >= min) & (att_m6_nofault['TimeUS'] <= max	))]
att_m6_nofault=att_m6_nofault.reset_index(drop=True)
print(att_m6_nofault)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      73598303     0.00  1.77     -0.55  -1.55   18.84  15.93
1      73609051     0.00  1.70     -0.54  -1.53   18.84  15.96
2      73619815     0.00  1.62     -0.53  -1.51   18.84  15.99
3      73630059     0.01  1.62     -0.53  -1.52   18.84  16.00
4      73640504     0.01  1.63     -0.53  -1.51   18.84  16.01
...         ...      ...   ...       ...    ...     ...    ...
6139  141327089    -0.61 -0.09     -2.50  -2.57    7.54   4.16
6140  141344718    -0.43 -0.16     -2.64  -2.48    7.54   3.98
6141  141354891    -0.25 -0.13     -2.76  -2.25    7.54   3.91
6142  141365784    -0.05 -0.15     -2.88  -2.06    7.54   3.89
6143  141376080     0.13 -0.24     -2.98  -1.93    7.54   3.86

[6144 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m6_nofault)>len(xkf1_m6_nofault):
    to_add=att_m6_nofault[len(xkf1_m6_nofault):]
    att_m6_nofault=att_m6_nofault[:len(xkf1_m6_nofault)]

if len(xkf1_m6_nofault)>len(att_m6_nofault):
    to_add=xkf1_m6_nofault[len(att_m6_nofault):]

for idx,i in enumerate(att_m6_nofault['Roll']):
    if(xkf1_m6_nofault['Roll'][idx] != i):
        att_m6_nofault['Roll'][idx] = ((att_m6_nofault['Roll'][idx] + xkf1_m6_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m6_nofault['Pitch']):
    if(xkf1_m6_nofault['Pitch'][idx] != i):
        att_m6_nofault['Pitch'][idx] = ((att_m6_nofault['Pitch'][idx] + xkf1_m6_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m6_nofault['Yaw']):
    if(xkf1_m6_nofault['Yaw'][idx] != i):
        att_m6_nofault['Yaw'][idx] = ((att_m6_nofault['Yaw'][idx] + xkf1_m6_nofault['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m6_nofault=pd.concat([att_m6_nofault,to_add])

print(att_m6_nofault)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      73598303     0.00  1.77     -0.55  -1.55   18.84  15.93
1      73609051     0.00  1.70     -0.54  -1.53   18.84  15.96
2      73619815     0.00  1.62     -0.53  -1.51   18.84  15.99
3      73630059     0.01  1.62     -0.53  -1.52   18.84  16.00
4      73640504     0.01  1.63     -0.53  -1.51   18.84  16.01
...         ...      ...   ...       ...    ...     ...    ...
6139  141327089    -0.61 -0.09     -2.50  -2.57    7.54   4.16
6140  141344718    -0.43 -0.16     -2.64  -2.48    7.54   3.98
6141  141354891    -0.25 -0.13     -2.76  -2.25    7.54   3.91
6142  141365784    -0.05 -0.15     -2.88  -2.06    7.54   3.89
6143  141376080     0.13 -0.24     -2.98  -1.93    7.54   3.86

[6144 rows x 7 columns]


C:\Users\mgale\AppData\Local\Temp\ipykernel_12576\2143887165.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m6_nofault['Yaw'][idx] = ((att_m6_nofault['Yaw'][idx] + xkf1_m6_nofault['Yaw'][idx])/2)


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m6_nofault = pd.read_csv("ESC_0.csv")
esc_0_m6_nofault = esc_0_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m6_nofault = esc_0_m6_nofault[((esc_0_m6_nofault['TimeUS'] >= min) & (esc_0_m6_nofault['TimeUS'] <= max))]
esc_0_m6_nofault=esc_0_m6_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m6_nofault = pd.read_csv("ESC_1.csv")
esc_1_m6_nofault = esc_1_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m6_nofault = esc_1_m6_nofault[((esc_1_m6_nofault['TimeUS'] >= min) & (esc_1_m6_nofault['TimeUS'] <= max))]
esc_1_m6_nofault=esc_1_m6_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m6_nofault = pd.read_csv("ESC_2.csv")
esc_2_m6_nofault = esc_2_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m6_nofault = esc_2_m6_nofault[((esc_2_m6_nofault['TimeUS'] >= min) & (esc_2_m6_nofault['TimeUS'] <= max))]
esc_2_m6_nofault=esc_2_m6_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m6_nofault = pd.read_csv("ESC_3.csv")
esc_3_m6_nofault = esc_3_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m6_nofault = esc_3_m6_nofault[((esc_3_m6_nofault['TimeUS'] >= min) & (esc_3_m6_nofault['TimeUS'] <= max))]
esc_3_m6_nofault=esc_3_m6_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m6_nofault = pd.read_csv("ESC_4.csv")
esc_4_m6_nofault = esc_4_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m6_nofault = esc_4_m6_nofault[((esc_4_m6_nofault['TimeUS'] >= min) & (esc_4_m6_nofault['TimeUS'] <= max))]
esc_4_m6_nofault=esc_4_m6_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m6_nofault = pd.read_csv("ESC_5.csv")
esc_5_m6_nofault = esc_5_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m6_nofault = esc_5_m6_nofault[((esc_5_m6_nofault['TimeUS'] >= min) & (esc_5_m6_nofault['TimeUS'] <= max))]
esc_5_m6_nofault=esc_5_m6_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m6_nofault = pd.read_csv("IMU_0.csv")
imu_0_m6_nofault = imu_0_m6_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m6_nofault = imu_0_m6_nofault[((imu_0_m6_nofault['TimeUS'] >= min) & (imu_0_m6_nofault['TimeUS'] <= max))]

imu_1_m6_nofault = pd.read_csv("IMU_1.csv")
imu_1_m6_nofault = imu_1_m6_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m6_nofault = imu_1_m6_nofault[((imu_1_m6_nofault['TimeUS'] >= min) & (imu_1_m6_nofault['TimeUS'] <= max))]

imu_2_m6_nofault = pd.read_csv("IMU_2.csv")
imu_2_m6_nofault = imu_2_m6_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m6_nofault = imu_2_m6_nofault[((imu_2_m6_nofault['TimeUS'] >= min) & (imu_2_m6_nofault['TimeUS'] <= max))]

imu_m6_nofault = pd.concat((imu_0_m6_nofault, imu_1_m6_nofault, imu_2_m6_nofault))
imu_m6_nofault=imu_m6_nofault.groupby(imu_m6_nofault.TimeUS, as_index=False).mean()

display(imu_m6_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,73587431,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330
1,73590478,0.092451,-0.148777,0.007550,-0.255108,-0.065997,-9.740719
2,73592092,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333
3,73598199,0.136942,-0.077143,-0.004878,-0.089322,0.035817,-9.557465
4,73604174,-0.115818,0.011950,0.043407,-0.420610,-0.265488,-9.875872
...,...,...,...,...,...,...,...
23374,141365515,-0.036255,0.351672,-0.038613,-1.075691,-0.784255,-9.579230
23375,141368756,-0.025718,0.381097,-0.022933,-1.067247,-0.849385,-9.791680
23376,141370810,-0.157157,0.354411,-0.023660,-1.086078,-0.746119,-9.873503
23377,141373732,-0.240294,0.277462,-0.019804,-1.017304,-0.481268,-10.041165


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m6_nofault = pd.merge_ordered(imu_m6_nofault,att_m6_nofault)
m6_nofault=m6_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m6_nofault = pd.merge_ordered(m6_nofault, esc_0_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_1_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_2_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_3_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_4_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_5_m6_nofault)
m6_nofault=m6_nofault.fillna(method="ffill").fillna(method="bfill")
m6_nofault = pd.merge_ordered(m6_nofault, rcou_m6_nofault)
m6_nofault=m6_nofault.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m6_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m6_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m6_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m6_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m6_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m6_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m6_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m6_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

6144
6725
7307
7885
8458
9029
9595


In [13]:
m6_nofault = m6_nofault[~m6_nofault.TimeUS.isin(index_to_remove)]
m6_nofault["TimeUS"] = m6_nofault["TimeUS"] - m6_nofault.iloc[0]["TimeUS"]
m6_nofault["TimeUS"] = m6_nofault["TimeUS"].astype(int)
m6_nofault["Guasto"] = guasto

In [14]:
from datetime import timedelta
m6_nofault=m6_nofault.reset_index(drop=True)
m6_nofault['TimeUS'] = pd.to_datetime(m6_nofault['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m6_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0
1,00:00:00.003047,0.092451,-0.148777,0.007550,-0.255108,-0.065997,-9.740719,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0
2,00:00:00.004661,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0
3,00:00:00.010768,0.136942,-0.077143,-0.004878,-0.089322,0.035817,-9.557465,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0
4,00:00:00.016743,-0.115818,0.011950,0.043407,-0.420610,-0.265488,-9.875872,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23374,00:01:07.778084,-0.036255,0.351672,-0.038613,-1.075691,-0.784255,-9.579230,-0.25,-0.13,-2.76,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0
23375,00:01:07.781325,-0.025718,0.381097,-0.022933,-1.067247,-0.849385,-9.791680,-0.05,-0.15,-2.88,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0
23376,00:01:07.783379,-0.157157,0.354411,-0.023660,-1.086078,-0.746119,-9.873503,-0.05,-0.15,-2.88,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0
23377,00:01:07.786301,-0.240294,0.277462,-0.019804,-1.017304,-0.481268,-10.041165,-0.05,-0.15,-2.88,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})','guasto'])
    for i in range(int(len(m6_nofault)/350)):
        V1=m6_nofault[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2,'guasto':guasto}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_12576\2823777097.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_12576\2823777097.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),guasto_x,...,guasto_y,<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),guasto
0,0.0,-0.033838,0.004603,-0.139153,0.075732,0.0,91.0,8.128651,1.628904,0.0,...,0.0,3.653419,0.497593,-0.290867,3.720705,78.0,76.0,26.125920,10.346820,0.0
1,1.0,-0.033680,0.004599,-0.138570,0.075633,0.0,60.0,8.122907,1.873474,0.0,...,0.0,3.654545,0.496622,-0.280129,3.721682,78.0,76.0,26.161171,10.082183,0.0
2,2.0,-0.033415,0.004611,-0.147460,0.075593,0.0,68.0,8.154549,2.154453,0.0,...,0.0,3.655666,0.495654,-0.269380,3.722653,78.0,76.0,26.196253,9.788830,0.0
3,3.0,-0.033303,0.004602,-0.143017,0.075487,0.0,92.0,8.131867,1.594375,0.0,...,0.0,3.656780,0.494689,-0.258622,3.723618,76.0,74.0,26.231090,9.468613,0.0
4,4.0,-0.033341,0.004590,-0.136083,0.075422,0.0,69.0,8.055514,1.916428,0.0,...,0.0,3.657887,0.493727,-0.247853,3.724577,78.0,76.0,26.265586,9.123674,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61.0,-0.033834,0.004468,-0.091229,0.074844,0.0,77.0,3.629050,2.351808,0.0,...,0.0,3.732597,0.460642,0.329663,3.793653,89.0,88.0,21.558010,9.807144,0.0
62,62.0,-0.033994,0.004467,-0.098090,0.074915,0.0,82.0,3.486271,1.884974,0.0,...,0.0,3.733511,0.459869,0.340586,3.794451,88.0,87.0,21.032784,9.985937,0.0
63,63.0,-0.034181,0.004471,-0.108344,0.075024,0.0,108.0,3.332299,1.832923,0.0,...,0.0,3.734420,0.459098,0.351515,3.795245,91.0,89.0,20.499580,10.153345,0.0
64,64.0,-0.034515,0.004507,-0.121542,0.075413,0.0,85.0,3.117207,2.777086,0.0,...,0.0,3.735325,0.458329,0.362450,3.796035,88.0,86.0,19.958815,10.308565,0.0


In [18]:
path_file = path_file.replace(r"M6\NO_FAULT6", "")
os.chdir(path_file)
m6_nofault.to_csv('m6_nofault.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
